In [1]:
# Rather than importing everything manually, we'll make things easy
#   and load them all in utils.py, and just import them from there.
%matplotlib inline
import utils; reload(utils)
from utils import *

Using gpu device 0: Tesla K80 (CNMeM is disabled)
Using Theano backend.


In [2]:
%matplotlib inline
from __future__ import division,print_function
import os, json
from glob import glob
import numpy as np
import scipy
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import confusion_matrix
np.set_printoptions(precision=4, linewidth=100)
from matplotlib import pyplot as plt
import utils; reload(utils)
from utils import plots, get_batches, plot_confusion_matrix, get_data

In [3]:
from numpy.random import random, permutation
from scipy import misc, ndimage
from scipy.ndimage.interpolation import zoom

import keras
from keras import backend as K
from keras.utils.data_utils import get_file
from keras.models import Sequential
from keras.layers import Input
from keras.layers.core import Flatten, Dense, Dropout, Lambda
from keras.layers.convolutional import Convolution2D, MaxPooling2D, ZeroPadding2D
from keras.optimizers import SGD, RMSprop
from keras.preprocessing import image

In [4]:
#path = "../data/dogsandcats_small/" # we copied a fraction of the full set for tests
path = "../data/dogsandcats/"
model_path = path + "models/"
if not os.path.exists(model_path):
    os.mkdir(model_path)
    print('Done')

In [5]:
from vgg16 import Vgg16
from resnet50 import Resnet50

In [6]:
batch_size = 100

In [7]:
def get_batches(dirname, gen=image.ImageDataGenerator(), shuffle=True, 
                batch_size=batch_size, class_mode='categorical'):
    return gen.flow_from_directory(path+dirname, target_size=(224,224), 
                class_mode=class_mode, shuffle=shuffle, batch_size=batch_size)

In [8]:
# Use batch size of 1 since we're just doing preprocessing on the CPU
val_batches = get_batches('valid', shuffle=False, batch_size=batch_size) # no shuffle as we store conv output
trn_batches = get_batches('train', shuffle=False, batch_size=batch_size) # no shuffle as we store conv output

Found 4000 images belonging to 2 classes.
Found 21000 images belonging to 2 classes.


In [9]:
val_batches.filenames[0:10]

['cat/cat.1262.jpg',
 'cat/cat.9495.jpg',
 'cat/cat.3044.jpg',
 'cat/cat.1424.jpg',
 'cat/cat.8210.jpg',
 'cat/cat.8847.jpg',
 'cat/cat.308.jpg',
 'cat/cat.10802.jpg',
 'cat/cat.5060.jpg',
 'cat/cat.10406.jpg']

In [10]:
val_labels = onehot(val_batches.classes)
trn_labels = onehot(trn_batches.classes)

In [11]:
'''import hashlib
def modelhash(mdl):
    chaine = str(mdl.to_json())
    return hashlib.md5(chaine).hexdigest()'''
# THE ABOVE FUNCTION DOES NOT WORK DUE TO LAYER DEFAULT NAMES

'import hashlib\ndef modelhash(mdl):\n    chaine = str(mdl.to_json())\n    return hashlib.md5(chaine).hexdigest()'

In [12]:
if True:
    realvgg = Vgg16()
    conv_layers, fc_layers = split_at(realvgg.model, Flatten)
    #conv_layers, fc_layers = split_at(realvgg.model, Convolution2D)
    conv_model = Sequential(conv_layers)
    conv_model_hash = 'conv_v3'

/home/ubuntu/anaconda2/lib/python2.7/site-packages/keras/layers/core.py:622: UserWarning: `output_shape` argument not specified for layer lambda_1 and cannot be automatically inferred with the Theano backend. Defaulting to output shape `(None, 3, 224, 224)` (same as input shape). If the expected output shape is different, specify it via the `output_shape` argument.
  .format(self.name, input_shape))


In [13]:
if False:
    vggbase = Vgg16()
    vggbase.model.pop()
    vggbase.model.pop()
    vggbase_hash = 'vgg_v1'
    # optional extra pop
    if False:
        vggbase.model.pop()
        vggbase.model.pop()
        #vggbase_hash = modelhash(vggbase.model)
        vggbase_hash = 'vgg_v2'
    print(vggbase_hash)

In [14]:
if False:
    resnetbase = Resnet50()
    resnetbase.model.layers.pop()
    for layer in resnetbase.model.layers:
        layer.trainable=False
    resnetbase.model = Model(resnetbase.model.input, resnetbase.model.layers[-1].output)
    resnetbase.model.compile(optimizer=RMSprop(lr=0.1), loss='categorical_crossentropy', metrics=['accuracy'])
    resnetbase_hash = 'resnet_v1'

### Will take a few minutes to complete the 1st time

In [15]:
if True:
    try:
        val_convfeatures = load_array(model_path+'valid_'+conv_model_hash+'_features.bc')
        if False: # force update
            raise
    except:
        print('Missing file')
        val_convfeatures = conv_model.predict_generator(val_batches, val_batches.nb_sample)
        save_array(model_path+'valid_'+conv_model_hash+'_features.bc', val_convfeatures)

In [16]:
if False:
    try:
        val_vggfeatures = load_array(model_path+'valid_'+vggbase_hash+'_features.bc')
        if False: # force update
            raise
    except:
        print('Missing file')
        val_vggfeatures = vggbase.model.predict_generator(val_batches, val_batches.nb_sample)
        save_array(model_path+'valid_'+vggbase_hash+'_features.bc', val_vggfeatures)

In [17]:
if False:
    try:
        val_resnetfeatures = load_array(model_path+'valid_'+resnetbase_hash+'_features.bc')
        if False: # force update
            raise
    except:
        print('Missing file')
        val_resnetfeatures = resnetbase.model.predict_generator(val_batches, val_batches.nb_sample)
        save_array(model_path+'valid_'+resnetbase_hash+'_features.bc', val_resnetfeatures)

### Will take a few minutes (maybe 10) to complete the 1st time

In [18]:
if True:
    try:
        trn_convfeatures = load_array(model_path+'train_'+conv_model_hash+'_features.bc')
        if False: # force update
            raise
    except:
        print('Missing file')
        trn_convfeatures = conv_model.predict_generator(trn_batches, trn_batches.nb_sample)
        save_array(model_path+'train_'+conv_model_hash+'_features.bc', trn_convfeatures)

In [19]:
if False:
    try:
        trn_vggfeatures = load_array(model_path+'train_'+vggbase_hash+'_features.bc')
        if False: # force update
            raise
    except:
        print('Missing file')
        trn_vggfeatures = vggbase.model.predict_generator(trn_batches, trn_batches.nb_sample)
        save_array(model_path+'train_'+vggbase_hash+'_features.bc', trn_vggfeatures)

In [20]:
if False:
    try:
        trn_resnetfeatures = load_array(model_path+'train_'+resnetbase_hash+'_features.bc')
        if False: # force update
            raise
    except:
        print('Missing file')
        trn_resnetfeatures = resnetbase.model.predict_generator(trn_batches, trn_batches.nb_sample)
        save_array(model_path+'train_'+resnetbase_hash+'_features.bc', trn_resnetfeatures)

### Ready to train the model

In [30]:
# see : https://github.com/fastai/courses/blob/master/deeplearning1/nbs/lesson3.ipynb

def proc_wgts(layer, ndo):
    # copy the weights from the pre-trained model
    # original weights are for a 50% drop out
    # we infer the corresponding weight for a new drop out (ndo) level
    return [w*0.5/(1.-ndo) for w in layer.get_weights()]

def get_fc_model(ndo):
    model = Sequential([
        Dense(4096, activation='relu', input_shape=conv_layers[-1].output_shape[1:]),
        Dropout(ndo),
        Dense(4096, activation='relu'),
        #BatchNormalization(),
        Dropout(ndo),
        #BatchNormalization(),
        Dense(2, activation='softmax')
        ])

    for l_new, l_orig in zip(model.layers[0:3], fc_layers[0:3]):
        assert (type(l_new) == type(l_orig))
        l_new.set_weights(proc_wgts(l_orig, ndo))
    
    for layer in model.layers[:-1]:
        layer.trainable = False
        
    model.layers[-1].trainable = True
    
    #opt = RMSprop(lr=0.00001, rho=0.7)
    opt = Adam()
    model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
    return model

### Train one or several models (ensembling)

In [62]:
ll_models = []
for i in range(1): # INFO : change here the size of the ensemble
    ll_models.append( get_fc_model(0.1) )
    #ll_models[-1].optimizer.lr = 1*1e-5
    ll_models[-1].fit(trn_convfeatures, trn_labels, validation_data=(val_convfeatures, val_labels), nb_epoch=10)

Train on 21000 samples, validate on 4000 samples
Epoch 1/10
21000/21000 [==============================] - 9s - loss: 0.0532 - acc: 0.9798 - val_loss: 0.0442 - val_acc: 0.9830
Epoch 2/10
21000/21000 [==============================] - 9s - loss: 0.0398 - acc: 0.9850 - val_loss: 0.0420 - val_acc: 0.9845
Epoch 3/10
21000/21000 [==============================] - 9s - loss: 0.0311 - acc: 0.9883 - val_loss: 0.0791 - val_acc: 0.9705
Epoch 4/10
21000/21000 [==============================] - 9s - loss: 0.0290 - acc: 0.9894 - val_loss: 0.0522 - val_acc: 0.9830
Epoch 5/10
21000/21000 [==============================] - 9s - loss: 0.0251 - acc: 0.9914 - val_loss: 0.0483 - val_acc: 0.9852
Epoch 6/10
21000/21000 [==============================] - 9s - loss: 0.0210 - acc: 0.9920 - val_loss: 0.0489 - val_acc: 0.9852
Epoch 7/10
21000/21000 [==============================] - 9s - loss: 0.0189 - acc: 0.9932 - val_loss: 0.0458 - val_acc: 0.9838
Epoch 8/10
21000/21000 [==============================] - 9s -

In [63]:
all_val_preds = []
for mdl in ll_models:
    these_val_preds = mdl.predict_on_batch(val_convfeatures)
    assert(len(these_val_preds) == 4000)
    all_val_preds.append( these_val_preds )
mean_val_preds = np.stack(all_val_preds).mean(axis=0)
categorical_accuracy(val_labels, mean_val_preds).eval()

array(0.9854999780654907, dtype=float32)

In [46]:
# WARNING : should save each model of the ensemble
#ll_model.save_weights(model_path+'llmodel_finetune1.h5')
#ll_model.load_weights(model_path+'llmodel_finetune1.h5')

In [33]:
test_batches = get_batches('test', shuffle=False, batch_size=batch_size, class_mode=None)
testfiles = test_batches.filenames
testfiles[0:10]

Found 12500 images belonging to 1 classes.


['test/10592.jpg',
 'test/7217.jpg',
 'test/3653.jpg',
 'test/4382.jpg',
 'test/2924.jpg',
 'test/10.jpg',
 'test/10916.jpg',
 'test/12374.jpg',
 'test/1871.jpg',
 'test/11645.jpg']

### Will take a few minutes (maybe 5) to complete the 1st time

In [34]:
try:
    test_convfeatures = load_array(model_path+'test_'+conv_model_hash+'_features.bc')
    #test_vggfeatures = load_array(model_path+'test_vggbase_features.bc')
    if False: # force update
        raise
except:
    print('Missing file')
    test_convfeatures = conv_model.predict_generator(test_batches, test_batches.nb_sample)
    save_array(model_path+'test_'+conv_model_hash+'_features.bc', test_convfeatures)
    #test_vggfeatures = vggbase.model.predict_generator(test_batches, test_batches.nb_sample)
    #save_array(model_path + 'test_vggbase_features.bc', test_vggfeatures)

Missing file


In [64]:
all_test_preds = []
for mdl in ll_models:
    these_test_preds = mdl.predict_on_batch(test_convfeatures)
    assert(len(these_test_preds) == 12500)
    all_test_preds.append( these_test_preds )
mean_test_preds = np.stack(all_test_preds).mean(axis=0)

In [67]:
mean_test_preds[0:10]

array([[  1.0000e+00,   2.9676e-10],
       [  1.0000e+00,   3.1055e-09],
       [  8.6116e-09,   1.0000e+00],
       [  1.0000e+00,   1.5027e-06],
       [  1.1490e-03,   9.9885e-01],
       [  1.0000e+00,   1.1879e-09],
       [  2.2946e-04,   9.9977e-01],
       [  1.0000e+00,   1.2871e-14],
       [  1.6734e-06,   1.0000e+00],
       [  1.2401e-08,   1.0000e+00]], dtype=float32)

In [57]:
dog_idx = 1
Z1 = [{'id':int(f.split('/')[-1].split('.')[0]), 'label':min(max(round(p[dog_idx],4),0.0001),0.9999)} 
      for f, p in zip(testfiles, mean_test_preds)]
def comp(x,y):
    return int(x['id']) - int(y['id'])
Z1 = sorted(Z1, comp)
Z1[0:18]

[{'id': 1, 'label': 0.9999},
 {'id': 2, 'label': 0.9999},
 {'id': 3, 'label': 0.9999},
 {'id': 4, 'label': 0.9999},
 {'id': 5, 'label': 0.0001},
 {'id': 6, 'label': 0.0001},
 {'id': 7, 'label': 0.0001},
 {'id': 8, 'label': 0.0001},
 {'id': 9, 'label': 0.0001},
 {'id': 10, 'label': 0.0001},
 {'id': 11, 'label': 0.0001},
 {'id': 12, 'label': 0.9999},
 {'id': 13, 'label': 0.0001},
 {'id': 14, 'label': 0.013},
 {'id': 15, 'label': 0.0001},
 {'id': 16, 'label': 0.0001},
 {'id': 17, 'label': 0.9999},
 {'id': 18, 'label': 0.9999}]

In [58]:
import csv

with open('predictions.csv', 'w') as csvfile:
    fieldnames = ['id', 'label']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    for z in Z1:
        writer.writerow(z)